# Привлекательность и уровень заработной платы
## Постановка задачи
По 1260 опрошенным имеются следующие данные:

* заработная плата за час работы, $;
* опыт работы, лет;
* образование, лет;
* внешняя привлекательность, в баллах от 1 до 5;
* бинарные признаки: пол, семейное положение, состояние здоровья (хорошее/плохое), членство в профсоюзе, цвет кожи (белый/чёрный), занятость в сфере обслуживания (да/нет).

Требуется оценить влияние внешней привлекательности на уровень заработка с учётом всех остальных факторов.


In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import pandas as pd
import scipy.stats as st
import seaborn as sns

In [ ]:
data = pd.read_csv('./beauty.csv', delimiter=';')
data.head()

Попарные диаграммы рассеяния всех количественных признаков: 

In [ ]:
sns.pairplot(data[['wage ', 'exper', 'educ', 'looks']])

## Предобработка

Посмотрим на распределение оценок привлекательности: 

In [ ]:
_ = plt.hist(data['looks'])

В группах looks=1 и looks=5 слишком мало наблюдений. Превратим признак looks в категориальный и закодируем с помощью фиктивных переменных:



In [ ]:
below_avg = 1.0*(data['looks']<3)
above_avg = 1.0*(data['looks']>3)
data = pd.concat([data, below_avg, above_avg], axis=1)

data.columns=list(data.columns[:-2])+['below_avg', 'above_avg']
data.head()


Распределение значений отклика:

In [ ]:
plt.hist(data['wage '])
plt.show()
plt.hist(np.log(data['wage '].values))

Один человек в выборке получает 77.72 долларов  в час, остальные — меньше 45 долларов; удалим этого человека.



In [ ]:
data = data[data['wage ']<70]

$\frac{max y}{min y} \approx 40>10$, поэтому найдём преобразование отклика методом Бокса-Кокса.

Возьмём $λ=0$, то есть, будем строить регрессию логарифма отклика.



In [ ]:
log_wage = st.boxcox(data['wage '], 0).reshape(-1,1)
old_columns = list(data.columns)
data2 = pd.concat([data.reset_index(drop=True), pd.Series(log_wage.flatten())], ignore_index=True, axis=1, join='outer')
data2.columns=list(old_columns)+['log_wage']
data2.head()


In [ ]:
plt.hist(log_wage)

# Модель 1

Построим линейную модель по всем признакам. Проверим остатки на нормальность.

In [ ]:
data2.columns

In [ ]:
import statsmodels.api as sm
model = sm.OLS.from_formula('log_wage~exper + union + goodhlth + black + female + married + service+ educ  + below_avg + above_avg', data2)
results = model.fit()
results.summary()

In [ ]:
data2.columns
features = data2.iloc[:, list(range(1,9))+list(range(10,12))]
features.head()

In [ ]:
print (st.shapiro(results.resid))
print (st.wilcoxon(results.resid))
from statsmodels.stats.diagnostic import het_breuschpagan
het_breuschpagan(results.resid, features)

In [ ]:
sns.pairplot(pd.concat([features[[ 'exper', 'educ']],results.resid],axis=1))

In [ ]:
st.probplot(results.resid, plot=plt)

In [ ]:
sns.residplot(features['exper'], results.resid,  
                          lowess=True, 
                          scatter_kws={'alpha': 0.5}, 
                          line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

В остатках наблюдается квадратичная зависимость от опыта работы.

# Модель 2

Добавим в модель 1 квадрат опыта работы.

In [ ]:
exp2 = features['exper']**2
old_columns = list(data2.columns)
data3 = pd.concat([data2.reset_index(drop=True), pd.Series(exp2)], ignore_index=True, axis=1, join='outer')
data3.columns=list(old_columns)+['exper2']
data3.head()


In [ ]:
import statsmodels.api as sm

model2 = sm.OLS.from_formula('log_wage~exper + union + goodhlth + black + female + married + service+ educ  + below_avg + above_avg + exper2', data3).fit()
model2.summary()

In [ ]:
data3.columns[list(range(1,9))+list(range(10, 12))+[13]]

In [ ]:
features = data3.iloc[:, list(range(1,9))+list(range(10, 12))+[13]]
features.head()

In [ ]:
print (st.shapiro(results.resid))
print (st.wilcoxon(results.resid))
from statsmodels.stats.diagnostic import het_breuschpagan
het_breuschpagan(results.resid, features)

Остатки ненормальны и гетероскедастичны

Незначимые признаки: здоровье, цвет кожи, семейное положение, привлекательность выше среднего. Прежде, чем удалять лишние признаки, проверим, не входят ли они в значимые попарные взаимодействия:



In [ ]:
formula = 'log_wage~' + '+'.join(features.columns)

In [ ]:
for f1 in range(len(features.columns)):
    for f2 in range(f1+1, len(features.columns)):
        formula2=#ваш код
        lm = sm.OLS.from_formula(formula2, data3).fit()
        for name, p in lm.pvalues.iteritems():
            if ':' in name and p<0.05:        
                print (name, p)

In [ ]:
model_all = sm.OLS.from_formula(formula, data3).fit()

anova = sm.stats.anova_lm(model_all, typ=1)
for id, p in enumerate(anova['PR(>F)']):
    if p<0.05:
        print(anova.iloc[id].name, p)

Визуальный анализ остатков не показывает никаких существенных особенностей: 

In [ ]:
sns.pairplot(pd.concat([features[[ 'exper', 'educ', 'exper2']],results.resid],axis=1))

In [ ]:
st.probplot(results.resid, plot=plt)

# Модель 3
Удалим из модели 2 незначимые признаки и добавим межфакторное взаимодействие пола и опыта работы.

In [ ]:
from statsmodels.formula.api import ols
model3 = ols('log_wage ~ exper + exper * female + female + union +  service + educ + above_avg + below_avg + exper2', data=data3).fit()
model3.summary()

In [ ]:
print (st.shapiro(model3.resid))
print (st.wilcoxon(model3.resid))


Значимы все признаки, кроме индикатора привлекательности выше среднего.

Визуальный анализ остатков не показывает никаких существенных особенностей: 

In [ ]:
sns.pairplot(pd.concat([features[[ 'exper', 'educ', 'exper2']],model3.resid],axis=1))

In [ ]:
st.probplot(model3.resid, plot=plt)

In [ ]:
model.summary()

In [ ]:
model2.summary()

In [ ]:
model2.compare_lr_test(model3)

In [ ]:
model2.compare_f_test(model3)

# Модель 4
Попробуем оставить в модели 2 цвет кожи и семейное положение, чтобы добавить их взаимодействия с полом. Как и в модели 3, добавим взаимодействие пола с опытом работы, а состояние здоровья удалим.

In [ ]:
from statsmodels.formula.api import ols
model4 = ols('log_wage ~ exper + exper2 + exper * female +  female + black + female * black + married + female * married + union + service + educ + above_avg + below_avg', data=data3).fit()
model4.summary()

In [ ]:
print (st.shapiro(model4.resid))
print (st.wilcoxon(model4.resid))


In [ ]:
sns.pairplot(pd.concat([features[[ 'exper', 'educ', 'looks']],model4.resid],axis=1))

In [ ]:
st.probplot(model4.resid, plot=plt)

In [ ]:
model4.summary()

# Модель 5


В предыдущей модели семейное положение незначимо; посмотрим, можно ли удалить его

In [ ]:
from statsmodels.formula.api import ols
model4 = ols('log_wage ~ exper + exper2 + exper * female +  female + black + female * black + married + female * married + union + service + educ + above_avg + below_avg', data=data3).fit(cov_type='HC0')
model4.summary()

In [ ]:
model4.wald_test_terms()

In [ ]:
model4.wald_test('married=female:married=0')

In [ ]:
from statsmodels.formula.api import ols
model5 = ols('log_wage ~ exper + exper2 + exper * female +  female + black + female * black + union + service + educ +  above_avg + below_avg ', data=data3).fit()
model5.summary()

In [ ]:
model4.compare_lr_test(model5), model4.compare_f_test(model5)

Модель получается значимо хуже. Удалим тогда только взаимодействие пола и семейного положения.



In [ ]:
from statsmodels.formula.api import ols
model5 = ols(' log_wage ~ exper + exper2 + exper * female +      female + black + female * black + married + union + service +       educ + above_avg + below_avg', data=data3).fit()
model5.summary()

In [ ]:
model4.compare_lr_test(model5), model4.compare_f_test(model5)

Модифицированный коэффициент детерминации убывает. Вернёмся к модели 4.



# Расстояние Кука
Посмотрим на влиятельные наблюдения: 
    

In [ ]:
cook = model4.get_influence().summary_frame().loc[:,'cooks_d']

In [ ]:
plt.scatter(data3['log_wage'], cook)
plt.xlim((0,4))
plt.ylim((-0.01,0.05))

Удалим наблюдения с расстоянием Кука больше 0.015 (порог выбран визуально) и перенастроим модель 4.

Сравним коэффициенты новой модели и модели 4:

In [ ]:
data4 = data3[cook<0.015]
data4.shape

In [ ]:
from statsmodels.formula.api import ols
model6 = ols('log_wage ~ exper + exper2 + exper * female +  female + black + female * black + married + female * married + union + service + educ + above_avg + below_avg', data=data4).fit()
model6.summary()

In [ ]:
model4.summary()

Итоговая модель (№6) объясняет 43% вариации логарифма отклика:

In [ ]:
plt.scatter(model6.predict(data4), data4['log_wage'])